<a href="https://colab.research.google.com/github/Alidev11/Alidev11/blob/main/SimpleStreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyarabic

     |████████████████████████████████| 126 kB 5.0 MB/s 


# **Import Libraries**

In [ ]:
import pyngrok
import streamlit as st
import pandas as pd
import plotly.express as px
!+ streamlit 
!+ pyngrok
!+ nohub/background

In [5]:
%%writefile app.py

###################### ---------------------------------------------- Import packages --------------------------------------------- ######################


#--------------------------------------------------------------------- Data Manipulation -------------------------------------------------------------
import numpy as np 
import pandas as pd
import re
import string
import io
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

#--------------------------------------------------------------------------- Model -------------------------------------------------------------------
from sklearn import datasets

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn import svm

#-------------------------------------------------------------------------- Visualisation ----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.express as px


#-------------------------------------------------------------------------- Streamlit ---------------------------------------------------------------------------
import pyngrok
import streamlit as st
#-------------------------------------------------------------------------- App Web Code ------------------------------------------------------------------------------------

st.set_page_config(page_title= 'Tweet Analyzer',
                   page_icon="🧊")
st.markdown(
            f"<h1 style='text-align: center; text-transform:uppercase; background: cadetblue; height: 12vh; display: flex; align-items: center; justify-content: center; '>Sentiment Analyzer</h1>", 
            unsafe_allow_html=True)




with st.form("my_form"):
    txt = st.text_area('Text to analyze', '''
  
     ''')

    # Every form must have a submit button.
    submitted = st.form_submit_button("Classify")
    
#------------------------------------------Importing Data-----------------------------------------
neg_train=pd.read_csv("/content/drive/MyDrive/TsvDataset/train_Arabic_tweets_negative_20190413.tsv", sep='\t', header=None, encoding='utf-8')
pos_train=pd.read_csv("/content/drive/MyDrive/TsvDataset/train_Arabic_tweets_positive_20190413.tsv", sep='\t', header=None, encoding='utf-8')
train_data= pd.concat([pos_train,neg_train],ignore_index=True)
train_data.columns=['label','tweet']
del pos_train,neg_train

neg_test=pd.read_csv("/content/drive/MyDrive/TsvDataset/test_Arabic_tweets_negative_20190413.tsv", sep='\t', header=None, encoding='utf-8')
pos_test=pd.read_csv("/content/drive/MyDrive/TsvDataset/train_Arabic_tweets_positive_20190413.tsv", sep='\t', header=None, encoding='utf-8')
test_data= pd.concat([pos_test,neg_test], ignore_index=True)
test_data.columns=['label','tweet']
del pos_test,neg_test

training_data=train_data.copy()
testing_data=test_data.copy()
data = pd.concat([training_data,testing_data], ignore_index=True)

#----------------------------------- Preprocess -----------------------------------------------
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
stop_words = stopwords.words()
diac = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

import pyarabic.araby as araby
def preprocess(text):
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    text = araby.strip_tashkeel(text)
    text = re.sub(diac, '', text)
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub(emoj, '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

training_data.tweet = training_data.tweet.apply(preprocess)
testing_data.tweet = testing_data.tweet.apply(preprocess)
data = pd.concat([training_data,testing_data], ignore_index=True)

#####----------------------------------------------- Data Variables #---------------------------------------------- ####
X_train = training_data.tweet.values
y_train = training_data.label.values

X_test = testing_data.tweet.values
y_test = testing_data.label.values
pipe = make_pipeline(TfidfVectorizer(),LogisticRegression())
model = pipe.fit(X_train,y_train)
#####------------------------------------------------- Submit Button #---------------------------------------------- ####

if submitted:
        txt = preprocess(txt)
        txt = [txt]
        y_pred = model.predict(txt)
        if y_pred[0] == 'neg':
          st.write("Negative")
        elif y_pred[0] == 'pos':
          st.write("Positive")

        

#####------------------------------------------------- CLASSIFICATION #---------------------------------------------- ####



Writing app.py


In [ ]:
!killall ngrok

ngrok: no process found


In [6]:
!ngrok authtoken 282MoXiHN5HL4bWV1iq4h1H86ul_3GsD3oEgZPCv1jb1SLxk5
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


2022-05-20 13:29:08.771 INFO    pyngrok.ngrok: Opening tunnel named: http-80-77c61221-f630-4cca-b3ea-734cf4e1fc3c
2022-05-20 13:29:08.834 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:08+0000 lvl=info msg="no configuration paths supplied"
2022-05-20 13:29:08.840 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:08+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-05-20 13:29:08.843 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:08+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-05-20 13:29:08.845 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:08+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2022-05-20 13:29:09.007 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:09+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2022-05-20 13:29:09.008 INFO    pyngrok.process.ngrok: t=2022-05-20T13:29:09+0000 lvl=info msg="client session established" obj=csess id=44175c23dfd5
2

<NgrokTunnel: "http://0768-104-196-49-86.ngrok.io" -> "http://localhost:80">

In [7]:
!streamlit run --server.port 80 app.py &>/dev/null&

In [ ]:
#------------------------------ Accuracy --------------------------------
        accuracy = model_selection.cross_val_score(model, X_train, y_train, cv=7, scoring='accuracy').mean()
        y_pred = model.predict(X_test)
        st.write("Accuracy ", round(accuracy, 3))

        #------------------------------ F1 Score --------------------------------
        f1 = make_scorer(f1_score, pos_label="pos")
        scoresk=model_selection.cross_val_score(model, X_train, y_train, cv=7, scoring=f1)
        f1_val = round(scoresk.mean(), 3)
        st.write(f"F1 score: {f1_val} +/- {scoresk.std():.3f}")

        #------------------------------ Precision --------------------------------
        precision = make_scorer(precision_score, pos_label="pos")
        scoresk=model_selection.cross_val_score(model, X_train, y_train, cv=7, scoring=precision)
        precision_val = round(scoresk.mean(), 3)
        st.write(f"Precision score: {precision_val} +/- {scoresk.std():.3f}")

        #------------------------------ Recall --------------------------------
        recall = make_scorer(recall_score, pos_label="pos")
        scoresk=model_selection.cross_val_score(model, X_train, y_train, cv=7, scoring=recall)
        recall_val = round(scoresk.mean(), 3)
        st.write(f"Recall score: {recall_val} +/- {scoresk.std():.3f}")

        plot_metrics(metrics)
